### Data visualization(2)

- EDA by location with coordinates
- reverse geocoding: longitude & lattitude => address


- Shp file
    - saved at 'bigquery-public-data.geo_us_boundaries.zip_codes'
    - if pickup_logitude, pickup_latitude belongs zip_code_geom, it's the location

In [7]:
import chart_studio.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
print(cf.__version__)
cf.go_offline()
%config InlineBackend.figure_format = 'retina'

0.17.3


In [8]:
# import GCP Client library
from google.cloud import bigquery

#instantiate a client for bigquery service
bigquery_client = bigquery.Client()

In [9]:
query = """
WITH base_data AS (
  SELECT 
    *
  FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
  WHERE EXTRACT(MONTH from pickup_datetime) = 1
), temp AS (
  SELECT nyc_taxi.*, gis.* 
  FROM (
    SELECT *
    FROM base_data
    WHERE pickup_latitude  <= 90 and pickup_latitude >= -90
    ) as nyc_taxi
  JOIN (
    SELECT zip_code, state_code, state_name, city, county, zip_code_geom
    FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
    WHERE state_code='NY'
    ) as gis 
  ON st_contains(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT 
  zip_code, 
  city,
  ST_ASTEXT(zip_code_geom) as zip_code_geom,
  DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
  count(*) as cnt
FROM temp
GROUP BY zip_code, city, zip_code_geom, pickup_hour
ORDER BY pickup_hour

"""

In [10]:
# run query and get data from bigquery
query_job = bigquery_client.query(query) #API request

# data into dataframe
df = query_job.to_dataframe()
df.head(100)

,zip_code,city,zip_code_geom,pickup_hour,cnt
0,11370,New York city,"MULTIPOLYGON(((-73.892656 40.793824, -73.89276...",2015-01-01,7
1,11367,New York city,"POLYGON((-73.84525 40.740166, -73.845211 40.74...",2015-01-01,3
2,10028,New York city,"POLYGON((-73.96316 40.777772, -73.96323 40.777...",2015-01-01,835
3,11216,New York city,"POLYGON((-73.957897 40.67087, -73.957974 40.67...",2015-01-01,37
4,10001,New York city,"POLYGON((-74.00828 40.750272, -74.008255 40.75...",2015-01-01,1221
...,...,...,...,...,...
95,10168,New York city,"POLYGON((-73.977677 40.752119, -73.978159 40.7...",2015-01-01,32
96,10473,New York city,"POLYGON((-73.884411 40.82297, -73.884396 40.82...",2015-01-01,1
97,10020,New York city,"MULTIPOLYGON(((-73.983263 40.761094, -73.98373...",2015-01-01,28
98,10111,New York city,"POLYGON((-73.97845 40.759065, -73.978039 40.75...",2015-01-01,28


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87020 entries, 0 to 87019
Data columns (total 5 columns):
zip_code         87020 non-null object
city             87020 non-null object
zip_code_geom    87020 non-null object
pickup_hour      87020 non-null datetime64[ns]
cnt              87020 non-null int64
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 3.3+ MB


### which zip_code has highest Trip 

In [12]:
zip_code_agg_df = df.groupby(['zip_code', 'zip_code_geom'])[['cnt']].sum().reset_index()

In [14]:
zip_code_agg_df.head(10)

,zip_code,zip_code_geom,cnt
0,10001,"POLYGON((-74.00828 40.750272, -74.008255 40.75...",627563
1,10002,"POLYGON((-73.997504 40.714069, -73.9973 40.713...",234527
2,10003,"POLYGON((-73.999366 40.731323, -73.999604 40.7...",704598
3,10004,"MULTIPOLYGON(((-74.018008 40.705935, -74.01801...",59700
4,10005,"POLYGON((-74.012508 40.706767, -74.0124 40.706...",52454
5,10006,"POLYGON((-74.015834 40.706984, -74.015956 40.7...",35708
6,10007,"POLYGON((-74.013382 40.714408, -74.0137 40.713...",87112
7,10009,"POLYGON((-73.988451 40.723277, -73.988643 40.7...",252903
8,10010,"POLYGON((-73.993739 40.741617, -73.994028 40.7...",387908
9,10011,"POLYGON((-74.012285 40.74387, -74.012359 40.74...",647831


In [15]:
zip_code_agg_df['percent'] = (zip_code_agg_df[['cnt']] / df[['cnt']]).round(3).rename(columns={'cnt':'percent'}).reset_index()['percent']

In [16]:
zip_code_agg_df.head(10)

,zip_code,zip_code_geom,cnt,percent
0,10001,"POLYGON((-74.00828 40.750272, -74.008255 40.75...",627563,89651.857
1,10002,"POLYGON((-73.997504 40.714069, -73.9973 40.713...",234527,78175.667
2,10003,"POLYGON((-73.999366 40.731323, -73.999604 40.7...",704598,843.830
3,10004,"MULTIPOLYGON(((-74.018008 40.705935, -74.01801...",59700,1613.514
4,10005,"POLYGON((-74.012508 40.706767, -74.0124 40.706...",52454,42.960
5,10006,"POLYGON((-74.015834 40.706984, -74.015956 40.7...",35708,35708.000
6,10007,"POLYGON((-74.013382 40.714408, -74.0137 40.713...",87112,87112.000
7,10009,"POLYGON((-73.988451 40.723277, -73.988643 40.7...",252903,84301.000
8,10010,"POLYGON((-73.993739 40.741617, -73.994028 40.7...",387908,32325.667
9,10011,"POLYGON((-74.012285 40.74387, -74.012359 40.74...",647831,323915.500
